In [ ]:
!git clone "https://github.com/ryanbaker1228/ENGR103_march_madness"

In [ ]:
# Pull from the git repo to make everything up to date
!cd ./ENGR103_march_madness && git pull && cd ../

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), 3.53 KiB | 3.53 MiB/s, done.
From https://github.com/ryanbaker1228/ENGR103_march_madness
   90bd4d3..ba69eb8  main       -> origin/main
Updating 90bd4d3..ba69eb8
Fast-forward
 MTeams.csv | 379 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 379 insertions(+)
 create mode 100644 MTeams.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm

In [ ]:
!ls ./ENGR103_march_madness/data

cbb13.csv  cbb16.csv  cbb19.csv  cbb22.csv  MNCAATourneyDetailedResults.csv
cbb14.csv  cbb17.csv  cbb20.csv  cbb23.csv  MNCAATourneySeeds.csv
cbb15.csv  cbb18.csv  cbb21.csv  cbb24.csv  MRegularSeasonDetailedResults.csv


In [ ]:
regular_season_df = pd.read_csv("./ENGR103_march_madness/data/MRegularSeasonDetailedResults.csv")
tournement_df = pd.read_csv("./ENGR103_march_madness/data/MNCAATourneyDetailedResults.csv")
seeds_df = pd.read_csv("./ENGR103_march_madness/data/MNCAATourneySeeds.csv")
seeds_2024 = pd.read_csv("./ENGR103_march_madness/data/2024_tourney_seeds.csv")

In [ ]:
print(regular_season_df.columns)

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')


In [ ]:
winning_team_cols: list = [
    "Season", "DayNum", "WTeamID", "WScore", "WLoc", "NumOT", "WFGM", "WFGA", "WFGM3",
    "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF", "LScore", "LTeamID"
]

losing_team_cols: list = [
    "Season", "DayNum", "LTeamID", "LScore", "WLoc", "NumOT", "LFGM", "LFGA", "LFGM3",
    "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF", "WScore", "WTeamID"
]
locations: dict = {
    "H": 1,
    "N": 0,
    "A": -1,
}

regular_season_results_df = pd.concat(
    [
        regular_season_df[winning_team_cols]
        .assign(Result=1)
        .rename(columns={
            "WTeamID": "TeamID",
            "LTeamID": "OppTeamID",
            "WScore":  "Score",
            "LScore":  "OppScore",
            "WLoc":    "Location",
            "WFGM":    "FGM",
            "WFGA":    "FGA",
            "WFGM3":   "FGM3",
            "WFGA3":   "FGA3",
            "WFTM":    "FTM",
            "WFTA":    "FTA",
            "WOR":     "OR",
            "WDR":     "DR",
            "WAst":    "Ast",
            "WTO":     "TO",
            "WStl":    "Stl",
            "WBlk":    "Blk",
            "WPF":     "PF",
        }),
        regular_season_df[losing_team_cols]
        .assign(Result=0)
        .rename(columns={
            "LTeamID": "TeamID",
            "WTeamID": "OppTeamID",
            "LScore":  "Score",
            "WScore":  "OppScore",
            "WLoc":    "Location",
            "LFGM":    "FGM",
            "LFGA":    "FGA",
            "LFGM3":   "FGM3",
            "LFGA3":   "FGA3",
            "LFTM":    "FTM",
            "LFTA":    "FTA",
            "LOR":     "OR",
            "LDR":     "DR",
            "LAst":    "Ast",
            "LTO":     "TO",
            "LStl":    "Stl",
            "LBlk":    "Blk",
            "LPF":     "PF",
        })
    ]
).reset_index(drop=True)

regular_season_results_df["ScoreDiff"] = (
    regular_season_results_df["Score"] - regular_season_results_df["OppScore"]
)

for index, row in regular_season_results_df.iterrows():
  regular_season_results_df.at[index, "Location"] = locations[
      regular_season_results_df.at[index, "Location"]
  ]

  if (row["Result"] == 0):
    regular_season_results_df.at[index, "Location"] *= -1

In [ ]:
regular_season_results_df.sample(6, random_state=42)

,Season,DayNum,TeamID,Score,Location,NumOT,FGM,FGA,FGM3,FGA3,...,DR,Ast,TO,Stl,Blk,PF,OppScore,OppTeamID,Result,ScoreDiff
67049,2016,33,1412,74,-1,0,28,56,9,22,...,28,19,15,8,4,16,61,1229,1,13
177869,2015,103,1313,70,-1,0,18,50,4,19,...,22,6,9,4,4,22,73,1288,0,-3
215172,2022,129,1438,43,0,0,18,52,3,11,...,29,14,11,5,6,11,63,1314,0,-20
204846,2020,101,1451,37,-1,0,13,40,3,14,...,24,7,21,1,0,11,72,1285,0,-35
52388,2013,68,1328,77,1,0,25,58,8,13,...,27,13,13,8,1,21,68,1329,1,9
109292,2024,40,1284,77,1,0,28,55,11,21,...,19,19,7,10,5,12,71,1351,1,6


In [ ]:
correlation = regular_season_results_df.corr()['Result'].sort_values(ascending=False)
print(correlation)

Result       1.000000e+00
ScoreDiff    7.931290e-01
Score        4.804380e-01
DR           3.922318e-01
Ast          3.734242e-01
FGM          3.681506e-01
FTM          3.025910e-01
Location     2.948143e-01
FTA          2.727116e-01
Blk          2.004081e-01
FGM3         1.869431e-01
Stl          1.638204e-01
TeamID       2.525718e-02
Season       1.183577e-12
DayNum       1.723721e-14
NumOT        1.080732e-15
OR          -1.357216e-02
OppTeamID   -2.525718e-02
FGA         -7.010063e-02
FGA3        -7.102541e-02
TO          -1.538344e-01
PF          -2.036141e-01
OppScore    -4.804380e-01
Name: Result, dtype: float64


In [ ]:
team_season_agg = (
    regular_season_results_df.groupby(["Season", "TeamID"])
    .agg(
        AvgScoreDiff = ("ScoreDiff", "mean"),
        MedScoreDiff = ("ScoreDiff", "median"),
        MinScoreDiff = ("ScoreDiff", "min"),
        MaxScoreDiff = ("ScoreDiff", "max"),
        NumWins = ("Result", lambda x: (x == 1).sum()),
        NumLosses = ("Result", lambda x: (x == 0).sum()),
        WinRate = ("Result", "mean"),
        DefRebounds = ("DR", "mean"),
        Assists = ("Ast", "mean"),
        FGM = ("FGM", "mean"),
        FTM = ("FTM", "mean"),
        FTA = ("FTA", "mean"),
        Block = ("Blk", "mean"),
        FGM3 = ("FGM3", "mean"),
        Steals = ("Stl", "mean"),
        Turnovers = ("TO", "mean"),
        PersonalFouls = ("PF", "mean"),
        OppScore = ("OppScore", "mean"),
        OffRebounds = ("OR", "mean"),
        FGA = ("FGA", "mean"),
        FGA3 = ("FGA3", "mean"),
    ).reset_index()
)
team_season_agg.head()

,Season,TeamID,AvgScoreDiff,MedScoreDiff,MinScoreDiff,MaxScoreDiff,NumWins,NumLosses,WinRate,DefRebounds,...,FTA,Block,FGM3,Steals,Turnovers,PersonalFouls,OppScore,OffRebounds,FGA,FGA3
0,2003,1102,0.250000,-3.0,-32,38,12,16,0.428571,16.821429,...,17.107143,1.785714,7.821429,5.964286,11.428571,18.750000,57.000000,4.178571,39.785714,20.821429
1,2003,1103,0.629630,-2.0,-16,33,13,14,0.481481,19.925926,...,25.851852,2.333333,5.444444,7.259259,12.629630,19.851852,78.148148,9.777778,55.851852,16.074074
2,2003,1104,4.285714,6.0,-19,28,17,11,0.607143,23.928571,...,20.928571,3.785714,6.357143,6.607143,13.285714,18.035714,65.000000,13.571429,57.178571,19.857143
3,2003,1105,-4.884615,-3.5,-42,34,7,19,0.269231,23.115385,...,21.846154,2.076923,7.576923,9.307692,18.653846,20.230769,76.653846,13.500000,61.615385,20.769231
4,2003,1106,-0.142857,-1.0,-26,26,13,15,0.464286,23.857143,...,16.464286,3.142857,6.107143,8.357143,17.035714,18.178571,63.750000,12.285714,55.285714,17.642857


In [ ]:
seeds_df["ChalkSeed"] = (
  seeds_df["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)

team_season_agg = team_season_agg.merge(
    seeds_df, how="left"
)
team_season_agg.columns

Index(['Season', 'TeamID', 'AvgScoreDiff', 'MedScoreDiff', 'MinScoreDiff',
       'MaxScoreDiff', 'NumWins', 'NumLosses', 'WinRate', 'DefRebounds',
       'Assists', 'FGM', 'FTM', 'FTA', 'Block', 'FGM3', 'Steals', 'Turnovers',
       'PersonalFouls', 'OppScore', 'OffRebounds', 'FGA', 'FGA3', 'Seed',
       'ChalkSeed'],
      dtype='object')

In [ ]:
winning_team_cols: list = [
    "Season", "WTeamID", "WScore", "LScore", "LTeamID"
]

losing_team_cols: list = [
    "Season", "LTeamID", "LScore", "WScore", "WTeamID"
]

tournement_results_df = pd.concat(
    [
        tournement_df[winning_team_cols]
        .assign(Result=1)
        .rename(columns={
            "WTeamID": "TeamID",
            "LTeamID": "OppTeamID",
            "WScore":  "Score",
            "LScore":  "OppScore",
        }),
        tournement_df[losing_team_cols]
        .assign(Result=0)
        .rename(columns={
            "LTeamID": "TeamID",
            "WTeamID": "OppTeamID",
            "LScore":  "Score",
            "WScore":  "OppScore",
        })
    ]
).reset_index(drop=True)

In [ ]:
tournement_features_df = tournement_results_df.merge(
    team_season_agg,
    on=["Season", "TeamID"],
    how="left"
).merge(
    team_season_agg
    .rename(columns={
        "TeamID": "OppTeamID",
        "AvgScoreDiff": "OppAvgScoreDiff",
        "MedScoreDiff": "OppMedScoreDiff",
        "MinScoreDiff": "OppMinScoreDiff",
        "MaxScoreDiff": "OppMaxScoreDiff",
        "NumWins": "OppNumWins",
        "NumLosses": "OppNumLosses",
        "WinRate": "OppWinRate",
        "DefRebounds": "OppDefRebounds",
        "Assists": "OppAssists",
        "FGM": "OppFGM",
        "FTM": "OppFTM",
        "FTA": "OppFTA",
        "Block": "OppBlock",
        "FGM3": "OppFGM3",
        "Steals": "OppSteals",
        "Turnovers": "OppTurnovers",
        "PersonalFouls": "OppPersonalFouls",
        "OppScore": "OppOppScore",
        "OffRebounds": "OppOffRebounds",
        "FGA": "OppFGA",
        "FGA3": "OppFGA3",
    }),
    on=["Season", "OppTeamID"],
)

In [ ]:
tournement_features_df["WinRateDiff"] = (
    tournement_features_df["WinRate"] - tournement_features_df["OppWinRate"]
)

In [ ]:
numeric_columns = tournement_features_df.select_dtypes(include=['number']).columns
correlation = tournement_features_df[numeric_columns].corr()['Result'].sort_values(ascending=False)
print(correlation)

Result              1.000000e+00
Score               4.824707e-01
ChalkSeed_y         3.787116e-01
WinRateDiff         3.274592e-01
AvgScoreDiff        2.889576e-01
NumWins             2.508874e-01
MedScoreDiff        2.452625e-01
WinRate             2.342486e-01
OppNumLosses        2.156973e-01
MaxScoreDiff        2.113932e-01
FGM                 1.781330e-01
MinScoreDiff        1.728962e-01
Block               1.452460e-01
Assists             1.425083e-01
FGA                 1.156779e-01
OppOppScore         1.044203e-01
OffRebounds         9.531219e-02
OppTurnovers        9.355462e-02
DefRebounds         7.878078e-02
Steals              6.961137e-02
OppPersonalFouls    6.142936e-02
FGM3                3.599315e-02
FGA3                1.094102e-02
FTM                 1.038187e-02
OppTeamID           9.644016e-03
OppFTA              2.931317e-03
Season             -1.023853e-14
FTA                -2.931317e-03
TeamID             -9.644016e-03
OppFTM             -1.038187e-02
OppFGA3   

In [ ]:
features = [
    "ChalkSeed_y",
    "ChalkSeed_x",
    "WinRateDiff",
    "OppAvgScoreDiff",
    "AvgScoreDiff",
    "OppWinRate",
]

target = "Result"

X = tournement_features_df[features]
y = tournement_features_df[target]
groups = tournement_features_df["Season"]
seasons = tournement_features_df["Season"].unique()

gkf = GroupKFold(n_splits=tournement_features_df["Season"].nunique())
cv_results = []
models = []

season_idx = 0
for train_index, test_index in gkf.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = xgb.XGBRegressor(
        eval_metric="logloss",
        n_estimators=1_000,
        learning_rate=0.001,
    )
    holdout_season = seasons[season_idx]
    print(f"Holdout Season: {holdout_season}")

    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

    y_pred = model.predict(X_test)
    score_ll = log_loss(y_test, y_pred)
    y_pred = y_pred > 0.5

    accuracy = accuracy_score(y_test, y_pred)
    cv_results.append(accuracy)
    season_idx += 1
    print(f"Season {holdout_season}: {accuracy} {score_ll}")
    models.append(model)

print("Average CV Accuracy:", np.mean(cv_results))

Holdout Season: 2003
[0]	validation_0-logloss:0.69301
[100]	validation_0-logloss:0.67967
[200]	validation_0-logloss:0.66886
[300]	validation_0-logloss:0.65970
[400]	validation_0-logloss:0.65281
[500]	validation_0-logloss:0.64681
[600]	validation_0-logloss:0.64313
[700]	validation_0-logloss:0.64045
[800]	validation_0-logloss:0.63953
[900]	validation_0-logloss:0.64033
[999]	validation_0-logloss:0.64234
Season 2003: 0.6268656716417911 0.6423448335335485
Holdout Season: 2004
[0]	validation_0-logloss:0.69291
[100]	validation_0-logloss:0.67126
[200]	validation_0-logloss:0.65251
[300]	validation_0-logloss:0.63660
[400]	validation_0-logloss:0.62448
[500]	validation_0-logloss:0.61417
[600]	validation_0-logloss:0.60628
[700]	validation_0-logloss:0.59941
[800]	validation_0-logloss:0.59316
[900]	validation_0-logloss:0.58847
[999]	validation_0-logloss:0.58430
Season 2004: 0.753731343283582 0.5843046825964789
Holdout Season: 2005
[0]	validation_0-logloss:0.69302
[100]	validation_0-logloss:0.68267
[2

In [ ]:
seeds_2024["ChalkSeed"] = (
    seeds_2024["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)

tourney_pairs = (
    seeds_2024.merge(seeds_2024, on=["Tournament"], suffixes=("", "Opp"))
    .assign(Season=2024)
    .query("TeamID != TeamIDOpp")
    .rename(columns={"Tournament": "League"})
)

tourney_pairs = (
    tourney_pairs.merge(
        team_season_agg[
            ["Season", "TeamID", "WinRate", "AvgScoreDiff"]
        ],
        on=["Season", "TeamID"],
        how="left",
    )
    .merge(
        team_season_agg[
            ["Season", "TeamID", "WinRate", "AvgScoreDiff"]
        ].rename(
            columns={
                "TeamID": "TeamIDOpp",
                "WinRate": "OppWinRate",
                "MedScoreDiff": "OppMedScoreDiff",
            }
        ),
        on=["Season", "TeamIDOpp"],
    )
    .reset_index(drop=True)
)

tourney_pairs["OppChalkSeed"] = (
    tourney_pairs["SeedOpp"]
    .str.replace("a", "")
    .str.replace("b", "")
    .str[1:]
    .astype("int")
)
tourney_pairs["WinRateDiff"] = (
    tourney_pairs["WinRate"] - tourney_pairs["OppWinRate"]
)
tourney_pairs = tourney_pairs.rename(columns={
    "ChalkSeed": "ChalkSeed_x",
    "OppChalkSeed": "ChalkSeed_y",
    "AvgScoreDiff_x": "AvgScoreDiff",
    "AvgScoreDiff_y": "OppAvgScoreDiff",
})

In [ ]:
for i, model in enumerate(models):
  tourney_pairs[f"pred_model{1}"] = model.predict(tourney_pairs[features])

tourney_pairs["Pred"] = tourney_pairs[
    [f for f in tourney_pairs.columns if "model" in f]
].mean(axis=1)

In [ ]:
clean_tourney_pairs = tourney_pairs.drop(columns=[
    "ChalkSeed_x",
    "WinRate",
    "AvgScoreDiff",
    "OppWinRate",
    "OppAvgScoreDiff",
    "ChalkSeed_y",
    "WinRateDiff",
    "pred_model1",
    "ChalkSeedOpp",
]).rename(columns={
    "SeedOpp": "OppSeed",
    "TeamIDOpp": "OppTeamID",
})

tourney_pairs["ID"] = (
    tourney_pairs["Season"].astype("str")
    + "_"
    + tourney_pairs["TeamID"].astype("str")
    + "_"
    + tourney_pairs["TeamIDOpp"].astype("str")
)

preds = clean_tourney_pairs.copy()

In [ ]:
round_slots = pd.read_csv(
    "./ENGR103_march_madness/data/MNCAATourneySlots.csv"
)
round_slots = round_slots[round_slots["Season"] == 2023]
round_slots = round_slots[
    round_slots["Slot"].str.contains("R")
]

seeds = pd.read_csv(
    "./ENGR103_march_madness/data/2024_tourney_seeds.csv"
)
seeds_m = seeds[seeds["Tournament"] == "M"]
seeds_w = seeds[seeds["Tournament"] == "W"]

preds["ID"] = preds["ID"].str.split("_")

In [ ]:
def prepare_data(seeds, preds):
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["Pred"]):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict

In [ ]:
def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    winners = []
    slots = []

    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning

        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1
result = run_simulation(
    brackets=n_brackets, seeds=seeds_m, preds=preds, round_slots=round_slots, sim=False
)

submission = pd.concat([result])
submission = submission.reset_index(drop=True)
submission.index.names = ["RowId"]
submission = submission.reset_index()
submission["Tournament"] = 'M'

100%|██████████| 1/1 [00:00<00:00, 397.45it/s]


In [ ]:
ss = pd.read_csv("./ENGR103_march_madness/sample_submission.csv")
submission[ss.columns] = submission[ss.columns]
submission[ss.columns].to_csv("./ENGR103_march_madness/submission.csv", index=False)

In [ ]:
submission_with_names = submission.rename(columns={"Team": "Seed"}).merge(
    seeds, on=["Seed", "Tournament"], how="left"
)

teams = pd.concat(
    [
        pd.read_csv("ENGR103_march_madness/MTeams.csv").assign(Tournament="M"),
    ]
)

submission_with_names = submission_with_names.merge(
    teams[["Tournament", "TeamID", "TeamName"]], how="left"
)

In [ ]:
submission_with_names.to_csv("submission_with_names.csv")
submission_with_names

,RowId,Bracket,Slot,Seed,Tournament,TeamID,TeamName
0,0,1,R1W1,W01,M,1163,Connecticut
1,1,1,R1W2,W02,M,1235,Iowa St
2,2,1,R1W3,W03,M,1228,Illinois
3,3,1,R1W4,W04,M,1120,Auburn
4,4,1,R1W5,W05,M,1361,San Diego St
...,...,...,...,...,...,...,...
58,58,1,R4Y1,Y02,M,1397,Tennessee
59,59,1,R4Z1,Z01,M,1222,Houston
60,60,1,R5WX,W01,M,1163,Connecticut
61,61,1,R5YZ,Z01,M,1222,Houston


In [ ]:
with open("ENGR103_march_madness/submission.m", 'w') as submission:
  for index, row in submission_with_names.iterrows():
    submission.write("{},M,1,{},{}".format(index, row["Slot"], row["Seed"]))

  submission.close()

In [ ]:
!cd ./ENGR103_march_madness/ && git add . && git commit -m "New write to submission" && git push && cd ..

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
